In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from keras.layers import Conv1D , Dropout , Flatten , MaxPooling1D, Dense, Input
from keras.layers.core import Lambda
from keras.models import Model , load_model
import random
import h5py

### Descarga y distribución de datos

In [2]:
# Cargamos los datos
path_local_data = '/home/anell/Desktop/Bovy/AnellExercises/Fits_files'
with h5py.File(f'{path_local_data}/apogeedr14_gaiadr2_with_spectrum_probando_rendimiento_1.h5') as F:  
    parallax = np.array(F['parallax'])
    parallax_error = np.array(F['parallax_err'])
    spectra = np.array(F['spectra'])
    Kmag = np.array(F['corrected_magnitude_K'])
    bp_rp = np.array(F['bp_rp'])
    Gmag = np.array(F['phot_g_mean_mag'])
    teff = np.array(F['NN_teff'])
    apogee_id = np.array(F['APOGEE_ID'])
    snr = np.array(F['SNR'])
    fe_h = np.array(F['Fe/H'])
    path_spectra = np.array(F['Path_spectra'])

/home/anell/anaconda3/envs/python37-astronn/lib/python3.7/site-packages/ipykernel_launcher.py:3: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
parallax.shape , spectra.shape , Kmag.shape

((57696,), (57696, 7514), (57696,))

In [4]:
cut_snr =[]
for i in range(len(snr)):
    if snr[i] > 20:
        cut_snr.append(i)
        
parallax = parallax[cut_snr]
parallax_error = parallax_error[cut_snr]
spectra = spectra[cut_snr]
Kmag = Kmag[cut_snr]
bp_rp = bp_rp[cut_snr]
Gmag = Gmag[cut_snr]
teff = teff[cut_snr]
apogee_id = apogee_id[cut_snr]
snr = snr[cut_snr]
fe_h = fe_h[cut_snr]
path_spectra = path_spectra[cut_snr]

In [5]:
parallax.shape, spectra.shape, Kmag.shape

((57565,), (57565, 7514), (57565,))

In [6]:
# Normalizamos Gmag , el color (G_bp - G_rp) y teff
Gmag_std = np.std(Gmag)
Gmag_mean = np.mean(Gmag)
Gmag_norm = (Gmag - Gmag_mean) / Gmag_std

bp_rp_std = np.std(bp_rp)
bp_rp_mean = np.mean(bp_rp)
bp_rp_norm = (bp_rp - bp_rp_mean) / bp_rp_std

teff_std = np.std(teff)
teff_mean = np.mean(teff)
teff_norm = (teff - teff_mean) / teff_std


#EStablecemos las variables que entrarán a la red y corregimos sus dimensiones
X = np.expand_dims(spectra,axis = 2)
Y = np.expand_dims(parallax,axis=1)
Y_error = np.expand_dims(parallax_error,axis=1)
K_mag = np.expand_dims(Kmag,axis=1)
G_mag = np.expand_dims(Gmag_norm,axis=1)
Bp_Rp = np.expand_dims(bp_rp_norm,axis=1)
Teff = np.expand_dims(teff_norm,axis=1)
Snr = np.expand_dims(snr,axis=1)
FeH = np.expand_dims(fe_h,axis=1)
Teff_without_norm = np.expand_dims(teff,axis=1)

X_offset = np.concatenate((G_mag, Bp_Rp , Teff), axis = 1) 

#Aleatorizamos la muestra
idx = []
for i in range(len(X)):
    idx.append(i)
random.seed(20)
random.shuffle(idx)
    
X = X[idx]                                       # shape: (15644, 7514 , 1)   
Y = Y[idx]                                       # shape: (15644, 1)  
K_mag = K_mag[idx]                               # shape: (15644, 1) 
X_offset = X_offset[idx]                         # shape: (15644, 3)
SNR = Snr[idx]                                   # shape: (15644, 1)
FeH = FeH[idx]                                   # shape: (15644, 1)
Teff_without_norm = Teff_without_norm[idx]       # shape: (15644, 1)

In [7]:
X.shape , Y.shape, parallax.shape

((57565, 7514, 1), (57565, 1), (57565,))

In [8]:
#Definición del modelo
def ApogeeDR14GaiaDR2(dim_t , dim_n): 
    """
    INPUT: 
    dim_t - number of time steps of spectrum 
    dim_n - number of features of spectrum
    """
    
    #SPECTRUM TO LUINOSITY
    dim_1 = 1 # number of corrected magnitude for one example 
    units = 1 #number of final output for one example
    inputs_spectra = Input(shape=(dim_t, dim_n)) 
    inputs_mag = Input(shape=(dim_1,), name="ApparentMagnitude-input")
    
    x_parallax = Conv1D(filters=4, kernel_size=2, activation='relu')(inputs_spectra)
    x_parallax = MaxPooling1D(pool_size=2)(x_parallax)
    x_parallax = Flatten()(x_parallax)
    x_parallax = Dense(164, activation='relu')(x_parallax) #relu
    x_parallax = Dense(164, activation='tanh')(x_parallax) #tanh
    x_parallax = Dense(64, activation='relu')(x_parallax) 
    x_parallax = Dense(64, activation='tanh')(x_parallax)
    x_parallax = Dense(32, activation='relu')(x_parallax)
    x_parallax = Dense(units, activation='softplus')(x_parallax)
    
    outputs_parallax = Lambda(lambda function: tf.math.multiply(function[0], tf.math.pow(10., 
                              tf.math.multiply(-0.2, function[1]))),
                              name='luminosity-to-parallax')([x_parallax, inputs_mag])
   
    #OFFSET CORRECTION : (optimization)
    inputs_offset = Input(shape=(3,), name="Offset-input")
    x_offset = Dense(64, activation='relu')(inputs_offset)
    x_offset = Dense(32, activation='relu')(x_offset) 
    x_offset = Dense(units, activation='tanh')(x_offset) 
    
    outputs_parallax_with_offset = Lambda(lambda function: tf.math.add(function[0], function[1]),
                                          name="Sum-parallax-offset")([outputs_parallax, x_offset]) 
    
    #Model setup
    model =  Model(inputs = [inputs_spectra,inputs_mag, inputs_offset],outputs = [outputs_parallax_with_offset])
    
    return model 

In [9]:
#Formación del set de entrenamiento
good_idx_snr = []
bad_idx_snr = []
for i in range(len(SNR)):
    if SNR[i] >= 200:           
        good_idx_snr.append(i)
    else:
        bad_idx_snr.append(i)

#SET ENTRENAMIENTO
X_train_snr = np.concatenate((X[good_idx_snr][:4000],X[bad_idx_snr][:2000]),axis = 0)
Y_train_snr = np.concatenate((Y[good_idx_snr][:4000],Y[bad_idx_snr][:2000]),axis = 0)
K_mag_train_snr = np.concatenate((K_mag[good_idx_snr][:4000],K_mag[bad_idx_snr][:2000]),axis = 0)
X_offset_train_snr = np.concatenate((X_offset[good_idx_snr][:4000],X_offset[bad_idx_snr][:2000]),axis = 0)

idx_random = []
for i in range(len(X_train_snr)):
    idx_random.append(i)
random.seed(1000)
random.shuffle(idx_random)

X_train_snr = X_train_snr[idx_random]
Y_train_snr = Y_train_snr[idx_random]
K_mag_train_snr = K_mag_train_snr[idx_random]
X_offset_train_snr = X_offset_train_snr[idx_random]

#SET DE VALIDACIÓN
#X_val_snr = np.concatenate((X[good_idx_snr][4000:5500],X[bad_idx_snr][2000:2500]),axis = 0)
#Y_val_snr = np.concatenate((Y[good_idx_snr][4000:5500],Y[bad_idx_snr][2000:2500]),axis = 0)
#K_mag_val_snr = np.concatenate((K_mag[good_idx_snr][4000:5500],K_mag[bad_idx_snr][2000:2500]),axis = 0)
#X_offset_val_snr = np.concatenate((X_offset[good_idx_snr][4000:5500],X_offset[bad_idx_snr][2000:2500]),axis = 0)

#idx_random_val = []
#for i in range(len(X_val_snr)):
#    idx_random_val.append(i)
#random.seed(5000)
#random.shuffle(idx_random_val)

#X_val_snr = X_val_snr[idx_random_val]
#Y_val_snr = Y_val_snr[idx_random_val]
#K_mag_val_snr = K_mag_val_snr[idx_random_val]
#X_offset_val_snr = X_offset_val_snr[idx_random_val]

In [10]:
n_timesteps, n_features = X_train_snr.shape[1], X_train_snr.shape[2]

Global_model = ApogeeDR14GaiaDR2(n_timesteps , n_features)

Global_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 7514, 1)]    0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 7513, 4)      12          input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    (None, 3756, 4)      0           conv1d[0][0]                     
__________________________________________________________________________________________________
flatten (Flatten)               (None, 15024)        0           max_pooling1d[0][0]              
_______________________________________________________________________________________

In [11]:
#Paré el entrenamiento antes para que no se sobreajustara
Global_model.compile(optimizer='adam', loss='mse', metrics=['mse'])
Global_model.fit([X_train_snr, K_mag_train_snr , X_offset_train_snr], Y_train_snr, epochs=300, batch_size=128, verbose=1)


Epoch 1/300
47/47 [==============================] - 2s 45ms/step - loss: 14.2433 - mse: 14.2433
Epoch 2/300
47/47 [==============================] - 2s 47ms/step - loss: 12.5736 - mse: 12.5736
Epoch 3/300
47/47 [==============================] - 2s 47ms/step - loss: 12.1484 - mse: 12.1484
Epoch 4/300
47/47 [==============================] - 2s 49ms/step - loss: 11.9231 - mse: 11.9231
Epoch 5/300
47/47 [==============================] - 2s 49ms/step - loss: 11.7959 - mse: 11.7959
Epoch 6/300
47/47 [==============================] - 2s 48ms/step - loss: 11.5803 - mse: 11.5803
Epoch 7/300
47/47 [==============================] - 2s 49ms/step - loss: 11.2329 - mse: 11.2329
Epoch 8/300
47/47 [==============================] - 2s 50ms/step - loss: 11.0414 - mse: 11.0414
Epoch 9/300
47/47 [==============================] - 2s 49ms/step - loss: 10.9285 - mse: 10.9285
Epoch 10/300
47/47 [==============================] - 2s 49ms/step - loss: 10.8987 - mse: 10.8987
Epoch 11/300
47/47 [=========

47/47 [==============================] - 3s 57ms/step - loss: 10.5009 - mse: 10.5009
Epoch 85/300
47/47 [==============================] - 3s 56ms/step - loss: 10.5107 - mse: 10.5107
Epoch 86/300
47/47 [==============================] - 3s 56ms/step - loss: 10.5023 - mse: 10.5023
Epoch 87/300
47/47 [==============================] - 2s 53ms/step - loss: 10.4963 - mse: 10.4963
Epoch 88/300
47/47 [==============================] - 2s 53ms/step - loss: 10.4918 - mse: 10.4918
Epoch 89/300
47/47 [==============================] - 3s 54ms/step - loss: 10.4942 - mse: 10.4942
Epoch 90/300
47/47 [==============================] - 3s 54ms/step - loss: 10.4971 - mse: 10.4971
Epoch 91/300
47/47 [==============================] - 2s 52ms/step - loss: 10.4967 - mse: 10.4967
Epoch 92/300
47/47 [==============================] - 3s 53ms/step - loss: 10.4848 - mse: 10.4848
Epoch 93/300
47/47 [==============================] - 3s 55ms/step - loss: 10.4828 - mse: 10.4828
Epoch 94/300
47/47 [=============

47/47 [==============================] - 2s 50ms/step - loss: 10.4405 - mse: 10.4405
Epoch 168/300
47/47 [==============================] - 2s 51ms/step - loss: 10.4232 - mse: 10.4232
Epoch 169/300
47/47 [==============================] - 2s 51ms/step - loss: 10.4208 - mse: 10.4208
Epoch 170/300
47/47 [==============================] - 2s 51ms/step - loss: 10.4295 - mse: 10.4295
Epoch 171/300
47/47 [==============================] - 2s 50ms/step - loss: 10.4216 - mse: 10.4216
Epoch 172/300
47/47 [==============================] - 2s 51ms/step - loss: 10.4092 - mse: 10.4092
Epoch 173/300
47/47 [==============================] - 2s 50ms/step - loss: 10.4313 - mse: 10.4313
Epoch 174/300
47/47 [==============================] - 2s 51ms/step - loss: 10.4249 - mse: 10.4249
Epoch 175/300
47/47 [==============================] - 2s 51ms/step - loss: 10.4128 - mse: 10.4128
Epoch 176/300
47/47 [==============================] - 2s 51ms/step - loss: 10.4142 - mse: 10.4142
Epoch 177/300
47/47 [===

47/47 [==============================] - 2s 49ms/step - loss: 10.3706 - mse: 10.3706
Epoch 250/300
47/47 [==============================] - 2s 51ms/step - loss: 10.3585 - mse: 10.3585
Epoch 251/300
47/47 [==============================] - 2s 50ms/step - loss: 10.3648 - mse: 10.3648
Epoch 252/300
47/47 [==============================] - 2s 52ms/step - loss: 10.3604 - mse: 10.3604
Epoch 253/300
47/47 [==============================] - 3s 55ms/step - loss: 10.3491 - mse: 10.3491
Epoch 254/300
47/47 [==============================] - 3s 55ms/step - loss: 10.4091 - mse: 10.4091: 0s - loss: 10.6046 - mse: 10.
Epoch 255/300
47/47 [==============================] - 2s 49ms/step - loss: 10.3723 - mse: 10.3723
Epoch 256/300
47/47 [==============================] - 2s 49ms/step - loss: 10.3986 - mse: 10.3986: 0s - loss: 10.0271 - 
Epoch 257/300
47/47 [==============================] - 2s 50ms/step - loss: 10.3694 - mse: 10.3694
Epoch 258/300
47/47 [==============================] - 2s 49ms/step -

In [18]:
Global_model.save('probando_rendimiento_1_SNR(4000-2000).h5')

In [ ]:
#Global_model.compile(optimizer='adam', loss='mse', metrics=['mse'])

#path_local = "/home/anell/Desktop/AnellExercises/Models/"

#callbacks = [ModelCheckpoint(f'{path_local}/model_probando_rendimiento_1_SNR(4000-2000)_2.h5', verbose=1, save_best_only=True)]

#Global_model.fit([X_train_snr,K_mag_train_snr,X_offset_train_snr], Y_train_snr, batch_size=128, #shuffle='batch', 
#                 epochs=300, callbacks=callbacks, 
#                 validation_data=([X_val_snr,K_mag_val_snr,X_offset_val_snr], Y_val_snr))